# 1. Skip-gram with naiive softmax 

* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture2.pdf
* https://arxiv.org/abs/1301.3781

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter
flatten = lambda l: [item for sublist in l for item in sublist]

In [2]:
print(torch.__version__)
print(nltk.__version__)

0.2.0+751198f
3.2.4


In [3]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [4]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [5]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if w in word2index.keys() else word2index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

def prepare_word(word,word2index):
    return Variable(LongTensor([word2index[word]]) if word in word2index.keys() else LongTensor([word2index["<UNK>"]]))

## Data load and Preprocessing 

### Load corpus : Gutenburg corpus

If you don't have gutenberg corpus, you can download it first using nltk.download()

In [6]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [7]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:100] # sampling sentences for test
corpus = [[word.lower() for word in sent] for sent in corpus]

### Extract Stopwords from unigram distribution's tails

In [8]:
word_count = Counter(flatten(corpus))
border =int(len(word_count)*0.01) 

In [9]:
stopwords = word_count.most_common()[:border]+list(reversed(word_count.most_common()))[:border]

In [10]:
stopwords = [s[0] for s in stopwords]

In [11]:
stopwords

[',', '.', 'the', 'of', 'and', 'genesis', 'moby', 'lins', 'eight', 'find']

### Build vocab

In [12]:
vocab = list(set(flatten(corpus))-set(stopwords))
vocab.append('<UNK>')

In [13]:
print(len(set(flatten(corpus))),len(vocab))

592 583


In [14]:
word2index = {'<UNK>' : 0} # for masking

for vo in vocab:
    if vo not in word2index.keys():
        word2index[vo]=len(word2index)

index2word = {v:k for k,v in word2index.items()} 

### Prepare train data 

In [15]:
WINDOW_SIZE = 3
windows =  flatten([list(nltk.ngrams(['<DUMMY>']*WINDOW_SIZE+c+['<DUMMY>']*WINDOW_SIZE,WINDOW_SIZE*2+1)) for c in corpus])

In [16]:
windows[0]

('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by')

In [17]:
train_data = []

for window in windows:
    for i in range(WINDOW_SIZE*2+1):
        if i==WINDOW_SIZE or window[i]=='<DUMMY>': continue
        train_data.append((window[WINDOW_SIZE],window[i]))

print(train_data[:WINDOW_SIZE*2])

[('[', 'moby'), ('[', 'dick'), ('[', 'by'), ('moby', '['), ('moby', 'dick'), ('moby', 'by')]


In [21]:
X_p=[]
y_p=[]

In [22]:
train_data[0]

('[', 'moby')

In [23]:
for tr in train_data:
    X_p.append(prepare_word(tr[0],word2index).view(1,-1))
    y_p.append(prepare_word(tr[1],word2index).view(1,-1))

In [24]:
train_data = list(zip(X_p,y_p))

In [25]:
len(train_data)

7606

### Modeling

In [26]:
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size,projection_dim):
        super(Skipgram,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim)
        self.embedding_u = nn.Embedding(vocab_size, projection_dim)

        self.embedding_v.weight.data.uniform_(-0.1, 0.1) # init
        self.embedding_u.weight.data.uniform_(-0.0, 0.0) # init
        #self.out = nn.Linear(projection_dim,vocab_size)
    def forward(self, center_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        outer_embeds = self.embedding_u(outer_words) # B x V x D
        scores = outer_embeds.bmm(center_embeds.transpose(1,2)) # BxVxD * BxDx1 => BxVx1
        
        return F.log_softmax(scores.squeeze(2))

    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds 

### Train  

In [27]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 100

In [28]:
losses = []
loss_function = nn.NLLLoss(ignore_index=0)
model = Skipgram(len(word2index),EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [29]:
for epoch in range(EPOCH):
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs) # B x 1
        targets = torch.cat(targets) # B x 1
        vocabs = prepare_sequence(list(vocab),word2index).expand(inputs.size(0),len(vocab))  # B x V
        model.zero_grad()

        pred = model(inputs,vocabs)
        loss = loss_function(pred,targets.view(-1))
        
        loss.backward()
        optimizer.step()
   
        losses.append(loss.data.tolist()[0])

    if epoch % 10==0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch,np.mean(losses)))
        losses=[]

Epoch : 0, mean_loss : 6.37
Epoch : 10, mean_loss : 6.14
Epoch : 20, mean_loss : 5.69
Epoch : 30, mean_loss : 5.35
Epoch : 40, mean_loss : 5.00
Epoch : 50, mean_loss : 4.66
Epoch : 60, mean_loss : 4.38
Epoch : 70, mean_loss : 4.17
Epoch : 80, mean_loss : 4.00
Epoch : 90, mean_loss : 3.87


### Test

In [30]:
def word_similarity(target,vocab):
    if USE_CUDA:
        target_V = model.prediction(prepare_word(target,word2index))
    else:
        target_V = model.prediction(prepare_word(target,word2index))
    similarities=[]
    for i in range(len(vocab)):
        if vocab[i] == target: continue
        
        if USE_CUDA:
            vector = model.prediction(prepare_word(list(vocab)[i],word2index))
        else:
            vector = model.prediction(prepare_word(list(vocab)[i],word2index))
        cosine_sim = F.cosine_similarity(target_V,vector).data.tolist()[0] 
        similarities.append([vocab[i],cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

In [36]:
test = random.choice(list(vocab))
test

'size'

In [37]:
word_similarity(test,vocab)

[['most', 0.8760154843330383],
 ['monstrous', 0.8514256477355957],
 ['bulk', 0.7815598845481873],
 ['breedeth', 0.77326899766922],
 ['worm', 0.6783211827278137],
 ['fish', 0.6459512710571289],
 ['biggest', 0.6388963460922241],
 ['d', 0.6097701191902161],
 ['wallen', 0.6066533923149109],
 ['touching', 0.6051964163780212]]